## Notenbonusprojekt
### Segmentierung von Stammkunden
** **

Für dieses Projekt liegen 200 Stammkundendaten vor. Es wird im Folgenden ein
Cluster-Algorithmus entwickelt, mit welchem dann die Segmentierung der Kunden
stattfindet. Daraus werden dann im Anschluss Handlungsempfehlungen abgeleitet.


In [4]:
#Zu Beginn werden die Kundendaten mit dem Package Pandas eingelesen; importiere
#noch weiter wichtige Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, metrics

pd = pd.read_csv('Kundendaten.csv', index_col= 0, delimiter=";", decimal=",")
pd


#Standardisierung
for col in pd.columns: #iteriert über alle Spalten im Datensatz
    pd['z_' + col] = (pd[col]-pd[col].mean())/pd[col].std()

X = pd[['z_geschlecht', 'z_alter','z_einkommen', 'z_monatsausgaben']].values #Variablen auswählen auf deren Basis klassifiziert werden soll
pd.head()



IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed